# Bayesian Search Algorithm

Use bayesian search based on the paper: "Bayesian-Based Search Decision Framework and Search Strategy Analysis in Probabilistic Search"

link: https://onlinelibrary.wiley.com/doi/full/10.1155/2020/8865381

In [152]:
# Import necessary libraries
import networkx as nx
import matplotlib.pyplot as plt

In [153]:
# Constants
map_graph_path = "../graphs/outputs/stricted_graph.graphml"

In [154]:
# Set target (ring) position (node)
def set_target_node(graph, target_node_id):
    nx.set_node_attributes(graph, {target_node_id: {"target": True}})

In [155]:
# Display graph information
def display_grpah_information(graph):
    # Print nodes data
    for node, data in graph.nodes(data=True):
        print(f"Node: {node}, Attributes: {data}")

    # Print edges data
    for u, v, data in graph.edges(data=True):
        print(f"Edge: ({u}, {v}), Attributes: {data}")

In [156]:
# Create an initial belief map
def initialize_belief_map(graph):
    for node in graph.nodes(data=True):
        nx.set_node_attributes(graph, {node[0]: {"P(target)": 0}})

In [157]:
# Read .graphml file
map_graph = nx.read_graphml(map_graph_path)

display_grpah_information(map_graph)


Node: n0, Attributes: {'x': '550.0', 'y': '800.0', 'shape_type': 'rectangle'}
Node: n1, Attributes: {'x': '500.0', 'y': '800.0', 'shape_type': 'rectangle'}
Node: n2, Attributes: {'x': '550.0', 'y': '750.0', 'shape_type': 'rectangle'}
Node: n3, Attributes: {'x': '600.0', 'y': '800.0', 'shape_type': 'rectangle'}
Node: n4, Attributes: {'x': '500.0', 'y': '750.0', 'shape_type': 'rectangle'}
Node: n5, Attributes: {'x': '550.0', 'y': '850.0', 'shape_type': 'rectangle'}
Node: n6, Attributes: {'x': '500.0', 'y': '850.0', 'shape_type': 'rectangle'}
Node: n7, Attributes: {'x': '600.0', 'y': '750.0', 'shape_type': 'rectangle'}
Node: n8, Attributes: {'x': '600.0', 'y': '850.0', 'shape_type': 'rectangle'}
Node: n9, Attributes: {'x': '550.0', 'y': '700.0', 'shape_type': 'rectangle'}
Node: n10, Attributes: {'x': '600.0', 'y': '700.0', 'shape_type': 'rectangle'}
Node: n11, Attributes: {'x': '500.0', 'y': '700.0', 'shape_type': 'rectangle'}
Node: n12, Attributes: {'x': '450.0', 'y': '700.0', 'shape_typ

In [ ]:
set_target_node(map_graph, "n20")   # Assign ring position to a desired node
initialize_belief_map(map_graph)    # Initialize belief map

In [159]:
display_grpah_information(map_graph)

Node: n0, Attributes: {'x': '550.0', 'y': '800.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n1, Attributes: {'x': '500.0', 'y': '800.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n2, Attributes: {'x': '550.0', 'y': '750.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n3, Attributes: {'x': '600.0', 'y': '800.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n4, Attributes: {'x': '500.0', 'y': '750.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n5, Attributes: {'x': '550.0', 'y': '850.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n6, Attributes: {'x': '500.0', 'y': '850.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n7, Attributes: {'x': '600.0', 'y': '750.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n8, Attributes: {'x': '600.0', 'y': '850.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n9, Attributes: {'x': '550.0', 'y': '700.0', 'shape_type': 'rectangle', 'P(target)': 0}
Node: n10, Attributes: {'x': '600.0', 'y': '700.0', 'shape_t